# Question 7: Time to Speak to an LLM

## 7(a) Installing Ollama

I will say, I'm a little nervous! Let's see how we do, I'll document this.

First of all, I use Arch BTW, so I looked through the Arch Linux package library, and found Ollama on there! So I'll run
```
~ >> sudo pacman -S ollama
```
to install it. It installed successfully. So next I'll try
```
~ >> ollama run phi3
Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it
```
Weird. Going to do some research to see if this is ok/expected. People complain a lot about Ollama online haha. I'm going to try again
```
~ >> ollama serve
~ >> ollama run phi3
pulling manifest 
pulling 633fc5be925f:  64% ▕██████████████████████████████████                    ▏ 1.4 GB/2.2 GB   13 MB/s     57s
```
And I'm seeing it download the model, as expected! The last 0.2GB are taking exceedingly long to download...
```
pulling manifest 
pulling 633fc5be925f: 100% ▕██████████████████████████████████████████████████████▏ 2.2 GB                         
pulling fa8235e5b48f: 100% ▕██████████████████████████████████████████████████████▏ 1.1 KB                         
pulling 542b217f179c: 100% ▕██████████████████████████████████████████████████████▏  148 B                         
pulling 8dde1baf1db0: 100% ▕██████████████████████████████████████████████████████▏   78 B                         
pulling 23291dc44752: 100% ▕██████████████████████████████████████████████████████▏  483 B                         
verifying sha256 digest 
writing manifest 
success 
>>> Send a message (/? for help)
```
Eventually, it downloaded, and I entered the REPL LLM chat environment. Now I can proceed with the task!
```
>>> I am wondering about the difference between programming and coding. Please explain.
Programming is a broader concept that encompinasses several aspects of creating software, from understanding 
user requirements to designing algorithms and writing code in various languages like Java or Python. It 
involves critical thinking skills such as problem-solving, logical reasoning, planning development strategy 
(like agile methodology), debugging the errors found during execution, maintaining a consistent coding style 
throughout projects, documentation of both code and project flow using tools like Javadoc for Java programs 
etc.

On the other hand, Coding is one specific part in programming where you actually write your program's source 
code based on previously established designs or algorithms. It involves writing clear, well-documented syntax 
that can be interpreted by a computer to perform tasks and solve problems as required. The focus here would 
primarily lie with understanding how different languages operate (syntax), using libraries effectively etc., 
while the broader aspects such as debugging techniques, planning strategies might not come into play often 
during this phase unless you're writing code for specific testing scenarios or optimizing a program to run 
faster which involves advanced programming knowledge.

Here is an example of how these two can interact in real-world scenario: A software engineer first understands 
the user requirements and creates algorithms (programming). Then, they write down corresponding source codes 
using their chosen language(coding), test them for errors or bugs, refine as necessary through iterations, 
document everything along with writing comments to explain certain sections of code. This continuous process 
embodies both programming and coding tasks working together in a harmonized manner throughout the software 
development lifecycle.

It's important to note that even though these terms are often used interchangeably, understanding their 
individual roles can help you identify where your skills might lie better or areas which need improvement for 
overall success as a programmer/coder and enhancing productivity in collaborative projects too!
```
Nice! Then I exited with ctrl+d, and had a go at running it again, and it worked without a hitch. Keen to move on to the next parts!

## 7(b) `ollama serve`

Because of the previous situation, I was pretty familiar with the `ollama serve` command. Used it without a problem, then I split my terminal to use the `curl` command.
```
~ >> curl http://127.0.0.1:11434/api/chat -d '{
  "model": "phi3",
  "messages": [
    {
      "role": "user",
      "content": "I am wondering about the difference between programming and coding. Please explain."
    }
  ],
  "stream": false}
}'
{"model":"phi3","created_at":"2025-08-12T13:21:45.730718459Z","message":{"role":"assistant","content":"Programming and coding are two terms that are often used interchangeably, but they do refer to different aspects of computer science. Let's explore both concepts further:\n\nCoding is simply writing instructions or commands in a specific programming language. These lines of code tell the computer what operations to perform based on certain conditions. Coders write scripts and programs using various languages like Python, Java, JavaScript, Ruby, etc., with syntax that computers can understand directly without any interpretation from humans.\n\nOn the other hand, programming refers to a broader concept which involves designing algorithms or plans for how different tasks should be carried out by computer systems - coding is just one component of this process and not necessarily required always. It also includes understanding hardware components (CPU architecture), software development methodologies like Agile Development etc., debugging code errors, testing programs to ensure that they are working as intended.\n\nSo in a nutshell: Coding can be seen as the actual writing aspect with syntax while programming is more of an art form which involves designing algorithms and plans for how various tasks should be carried out by computer systems - coding just being one component among these many things."},"done_reason":"stop","done":true,"total_duration":27198017810,"load_duration":7388548964,"prompt_eval_count":23,"prompt_eval_duration":762032992,"eval_count":241,"eval_duration":19044252754}
```
Great success!

Next,
```
~ >> man curl
[lots of irrelevent stuff...]
       -d, --data <data>
              (HTTP  MQTT)  Send  the  specified  data in a POST request to the HTTP server, in the same way that a
              browser does when a user has filled in an HTML form and presses the submit button.
```
So, in my own words, the `-d` flag says that the following data (namely the JSON `{"model":"phi3"...etc}`) should be sent to the HTTP server (the address `http:/127.0.0.1:11434/api/chat`) as a POST request. I've often used `curl` to download files from websites, but in this instance curl has sent this data to the server, and then waits for a response. The server receives that data, does what ever it wants to do with it (in this case, ships it to an LLM according to the data) and then sends back its response. `127.0.0.1` in this case is the IP address of the local host, my machines local IP address, and `11434` is the port which ollama is running from. 

## 7(c) Automating our `curl` with Julia, HTTP, and JSON

I started by going to my Julia package manager and adding `JSON` and `HTTP`. Turns out I already had `JSON`, but `HTTP` was downloaded successfully!

Most languages have something like `argv*` and `argc` to access the commandline arguments, and knowing that I wanted to make a commandline tool (CLI), I learned about Julia's convenient `ARGS` global variable. 

My first iteration recieved the ARGS, joined them with a space delimiter, and then echoed that message back to the user. Everything was working as expected, so I was ready to add the HTTP request.

I think investigating curl's `-d` flag gave me a strong hint that I want to use `HTTP.post()`! It felt pretty cool to make that connection. I sent a POST request to the server with no body written yet, and I saw the server recieve it and send back an error 400! Progress! I learned and applied how to write `Dict`s and then convert them to JSONs using `JSON.json()`. So I'm ready to put my POST data body together: the model, the messages (which itself has a role and a message subfield) and stream set to false (copying how the command given to me structured it.)

For a while, I was having an issue where the server was receiving my request, but bouncing it with error 500, invalid header. I found out that I need to tell the server how I expect my content to be returned to me, namely `"Content-Type" => "application/json"`. Once I added that to my POST request, the server was receiving my request, giving to the LLM, and sending it back to me!

The last issue I needed to fix was `return response` was giving me an error when I tried to print it. That's how I found out that `HTTP.post()` returns an object, and I need to access the body of the response and convert it to a String to actually get it printing to my console.

My code, and a sample usage:

In [4]:
using HTTP
using JSON

function askPhi(msg::String)::String
  response = HTTP.post(
    "http://127.0.0.1:11434/api/chat",
    ["Content-Type" => "application/json"],
    JSON.json(Dict(
      "model" => "phi3",
      "messages" => [Dict(
          "role" => "user",
          "content" => "$msg"
        )],
      "stream" => false
    ))
  )
  
  return String(response.body)
end

#https://docs.julialang.org/en/v1/manual/faq/#man-scripting
if abspath(PROGRAM_FILE) == @__FILE__
  println(askPhi(join(ARGS, " ")))
end

```
~ >> julia ./AskPhi.jl Hi Phi! How are you today? Can you explain to me how to integrate x squared between 0 and
 1?
{"model":"phi3","created_at":"2025-08-12T14:30:26.663732815Z","message":{"role":"assistant","content":"Hello there! I'm an AI, so I don't have feelings, but thanks for asking. Now let’s tackle your calculus question: To find the integral of \\(x^2\\) from 0 to 1, we first need to determine its indefinite integral and then apply the Fundamental Theorem of Calculus by evaluating it at the upper limit (which is x=1) and subtracting its value at the lower limit (which is x=0).\n\nFirstly, let’s find the antiderivative or indefinite integral of \\(x^2\\), which can be done using a power rule. The general form for this would be \\(\\frac{x^{n+1}}{n+1}\\), where n = 2 (since our function is x squared). So, we have:\n\n∫x² dx = \\frac{x³}{3} + C\n\nwhere 'C' represents the constant of integration. However, since you’re interested in a definite integral from 0 to 1, this ‘+C‘ will not be needed later on as it cancels out when subtracting these two bounds. Now we need to evaluate \\(\\frac{x³}{3}\\) at x=1 and x=0:\n\nAt x = 1, the value of our function is \\(\\frac{1^3}{3} = \\frac{1}{3}\\). At x = 0, this equals zero (as any number multiplied by zero equals zero). Therefore, we subtract these two values to find that the integral from 0 to 1 for \\(x^2\\) yields a result of \\(\\frac{1}{3}\\) - 0 = \\boxed{\\frac{1}{3}}.\n\nTo clarify further: when you have an indefinite integral (antiderivative) and want to evaluate it between two limits, the Fundamental Theorem of Calculus tells us that if F(x) is any antiderivative for f(x), then \\(\\int_{a}^{b}f(x)dx=F(b)-F(a)\\). In this case we used our indefinite integral as F(x) and evaluated it at x = 1 (upper bound) and subtracted its value at the lower limit, which was zero."},"done_reason":"stop","done":true,"total_duration":84176461297,"load_duration":235218398,"prompt_eval_count":36,"prompt_eval_duration":1413813312,"eval_count":490,"eval_duration":43017804380}
```

## 7(d) Basically ChatGPT

Because of how I formulated my previous question, I can import it as a package and use that functionality without duplicating my code! I actually had so much fun building this haha. My code and a sample of input/output:

In [3]:
include("AskPhi.jl")

function main()::Integer
  println("
   █████████           █████      ███████████  █████       ███ 
  ███░░░░░███         ░░███      ░░███░░░░░███░░███       ░░░  
 ░███    ░███   █████  ░███ █████ ░███    ░███ ░███████   ████ 
 ░███████████  ███░░   ░███░░███  ░██████████  ░███░░███ ░░███ 
 ░███░░░░░███ ░░█████  ░██████░   ░███░░░░░░   ░███ ░███  ░███ 
 ░███    ░███  ░░░░███ ░███░░███  ░███         ░███ ░███  ░███ 
 █████   █████ ██████  ████ █████ █████        ████ █████ █████
░░░░░   ░░░░░ ░░░░░░  ░░░░ ░░░░░ ░░░░░        ░░░░ ░░░░░ ░░░░░ 
")
  println("Welcome to the AskPhi REPL environment!\nWrite a message and press ENTER to send it to the Phi3 LLM!\nPress ENTER with no message to exit this program!\n")
  
  while true
    print("AskPhi >> ")
    input = readline()
    if input == ""
      break
    end
    response = JSON.parse(askPhi(input))
    println("\n$(response["message"]["content"])\n")
  end

  return 0
end

if abspath(PROGRAM_FILE) == @__FILE__
  main()
end

```
~ >> julia ./dev/AskPhiREPL.jl

   █████████           █████      ███████████  █████       ███ 
  ███░░░░░███         ░░███      ░░███░░░░░███░░███       ░░░  
 ░███    ░███   █████  ░███ █████ ░███    ░███ ░███████   ████ 
 ░███████████  ███░░   ░███░░███  ░██████████  ░███░░███ ░░███ 
 ░███░░░░░███ ░░█████  ░██████░   ░███░░░░░░   ░███ ░███  ░███ 
 ░███    ░███  ░░░░███ ░███░░███  ░███         ░███ ░███  ░███ 
 █████   █████ ██████  ████ █████ █████        ████ █████ █████
░░░░░   ░░░░░ ░░░░░░  ░░░░ ░░░░░ ░░░░░        ░░░░ ░░░░░ ░░░░░ 

Welcome to the AskPhi REPL environment!
Write a message and press ENTER to send it to the Phi3 LLM!
Press ENTER with no message to exit this program!

AskPhi >> Hi!

Hello there! How can I assist you today?

AskPhi >> How many marks do you think I'll get for this question? Ball park it.

As an artificial intelligence, I can provide a hypothetical range based on typical responses and the clarity of your answer to similar questions in general knowledge tests or educational settings: if your response is well-considered, accurate, concise, clear, free from plagiarism (if applicable), grammatically correct, and appropriately detailed for its level, you might score between 15% to 90%. The exact mark would depend on the specific criteria used by graders.

Ball park it: I'd estimate your answer could receive a grade of B-minus if well executed or maybe even an A-minus in case there are some minor errors, but overall excellent work considering you understand and can effectively communicate complex information clearly and concisely.

AskPhi >> Can you breifly explain how to prove two topologies are homeomorphic?

Two topological spaces $(X, \tau_1)$ and $(Y, \tau_2)$ are said to be homeomorphic or topologically equivalent if there exists a continuous bijective function $f: X \rightarrow Y$ whose inverse function $f^{-1}: Y \rightarrow X$ is also continuous. This condition preserves the structure of spaces in terms of topology and means that both $(X, \tau_1)$ and $(Y, \tau extraneous information about distances or angles between points) are essentially indistinguishable topologically.

If you can establish all these properties simultaneously, then $(X, \tau_1)$ is homeomorphic to $(Y, \tau_2)$, hence they are topologically equivalent.

AskPhi >> 
~ >> 
```

Works exactly as intended! Of course this is super barebones, and there's a lot that could be added to this (handling EOF/Ctrl+C cleaning, HTTP timeouts and errors, try/catch blocks in my askPhi function, useful error reporting, a memory/conversation state, saving conversations, expanding support for LLMs beyond Phi3... So much)  But I think this is sufficent for now!